In [95]:
#Spacy's algorithm
import nltk
import random, math, copy, os, string
from scipy.stats import pearsonr
from nltk.metrics import jaccard_distance
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn  
from nltk.corpus import wordnet_ic
from nltk.wsd import lesk
import en_core_web_sm
nlp = en_core_web_sm.load()
import re
from google.colab import drive
drive.mount('/content/drive')
import spacy
nltk.download('wordnet')
nltk.download('universal_tagset') 
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('wordnet_ic')
nltk.download('averaged_perceptron_tagger')
!pip install svgling

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet_ic to /root/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already 

In [91]:
import pandas as pd
dt = pd.read_csv('/content/drive/My Drive/content/drive/test-gold/STS.input.SMTeuroparl.txt',sep='\t',header=None)
dt['gs'] = pd.read_csv('/content/drive/My Drive/content/drive/test-gold/STS.gs.SMTeuroparl.txt',sep='\t',header=None)
dt = dt.rename(columns={0: 'ENT_A', 1: 'ENT_B'})
print(dt.head())
nltk.download('averaged_perceptron_tagger')
nltk.download('words')

                                               ENT_A  ...    gs
0  The leaders have now been given a new chance a...  ...  4.50
1  Amendment No 7 proposes certain changes in the...  ...  5.00
2  Let me remind you that our allies include ferv...  ...  4.25
3        The vote will take place today at 5.30 p.m.  ...  4.50
4  The fishermen are inactive, tired and disappoi...  ...  5.00

[5 rows x 3 columns]
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [92]:
# dictionary of all the NEC in the sentences
def get_NEC_dict(df_column):
  d = dict()
  for idxA, xA in enumerate(df_column): 
    sent = nlp(xA)
    for s in sent.ents:
      d[s.text] = s.label_
  return d

# replacing the words with their corresponding NEC
# regex + previously made dictionary
def get_NEC_REPL_sent(df_column, DICT):
  l=[]
  for sent in df_column:
    rep = dict((re.escape(k), v) for k, v in DICT.items()) 
    pattern = re.compile("|".join(rep.keys()))
    textf = pattern.sub(lambda m: rep[re.escape(m.group(0))], sent)
    l.append(textf)
  return l

#jaccard distance for wanted cols
def get_jd(data_f, colA, colB,name):
  data_f[name] = ""
  for index in data_f.index:
    data_f[name][index] = 1 - jaccard_distance(set(data_f[colA][index]), set(data_f[colB][index]))
  return data_f

def sent_word_set(df, col):
  df[col+"_tok"] = df[col].apply(lambda x: nltk.sent_tokenize(x))
  df[col+"_tok"] = df[col+"_tok"].apply(lambda x: nltk.word_tokenize(x[0]))
  df[col+"_tok"] = df[col+"_tok"].apply(set)

  return df

In [93]:
# Mapping the NEC to their words 
ent_a_dict = get_NEC_dict(dt['ENT_A'])
ent_b_dict = get_NEC_dict(dt['ENT_B'])

# Applying the replacement
dt['ENT_A_REPL'] = get_NEC_REPL_sent(dt['ENT_A'],ent_a_dict)
dt['ENT_B_REPL'] = get_NEC_REPL_sent(dt['ENT_B'],ent_b_dict)

#sent / word tokenizing and making a set (FOR THE REPLACED SENTENCES)
dt = sent_word_set(dt,'ENT_A_REPL')
dt = sent_word_set(dt,'ENT_B_REPL')

# same as above but for the original sentences 
dt = sent_word_set(dt,'ENT_A')
dt = sent_word_set(dt,'ENT_B')

# jaccard distance between the original sentence and the replaced sentence 
dt = get_jd(dt, 'ENT_A_tok','ENT_A_REPL_tok','JD_A_REPL')
dt = get_jd(dt, 'ENT_B_tok','ENT_B_REPL_tok','JD_B_REPL')


#jaccard ditance between NEC replaced sentences
dt = get_jd(dt, 'ENT_A_REPL_tok','ENT_B_REPL_tok','JD_ORIG_REPL')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [97]:
# pearson with: NEC being replaced in the original sentence and Sent tokenize + Word tokenize
round(pearsonr(dt['gs'], dt['JD_ORIG_REPL'])[0],2)

0.33

In [94]:
dt.head()

,ENT_A,ENT_B,gs,ENT_A_REPL,ENT_B_REPL,ENT_A_REPL_tok,ENT_B_REPL_tok,ENT_A_tok,ENT_B_tok,JD_A_REPL,JD_B_REPL,JD_ORIG_REPL
0,The leaders have now been given a new chance a...,The leaders benefit aujourd' hui of a new luck...,4.50,The leaders have now been given a new chance a...,The leaders benefit aujourd' hui of a new luck...,"{us, they, have, chance, ., been, let, The, no...","{of, them, ., ', luck, let, aujourd, The, ther...","{us, they, have, chance, ., been, let, The, no...","{of, them, ., ', luck, let, aujourd, The, ther...",1,1,0.346154
1,Amendment No 7 proposes certain changes in the...,Amendment No 7 is proposing certain changes in...,5.00,Amendment No 7 proposes certain changes in the...,Amendment No CARDINAL is proposing certain cha...,"{changes, Amendment, 7, certain, ., in, the, r...","{changes, Amendment, ., in, the, references, p...","{changes, Amendment, 7, certain, ., in, the, r...","{changes, Amendment, 7, ., in, the, references...",1,0.857143,0.666667
2,Let me remind you that our allies include ferv...,I would like to remind you that among our alli...,4.25,Let me remind you that our allies include ferv...,I would like to remind you that among our alli...,"{remind, of, our, allies, ., me, include, that...","{you, of, among, would, our, allies, are, I, s...","{remind, of, our, allies, ., me, include, that...","{you, of, among, would, our, allies, are, I, s...",1,1,0.391304
3,The vote will take place today at 5.30 p.m.,The vote will take place at 5.30pm,4.50,The vote will take place DATE at TIME,The vote will take place at CARDINAL,"{take, vote, DATE, at, will, The, place, TIME}","{take, vote, at, will, The, place, CARDINAL}","{take, vote, today, ., at, 5.30, will, The, pl...","{take, vote, 5.30pm, at, will, The, place}",0.5,0.75,0.666667
4,"The fishermen are inactive, tired and disappoi...","The fishermen are inactive, tired and disappoi...",5.00,"The fishermen are inactive, tired and disappoi...","The fishermen are inactive, tired and disappoi...","{fishermen, ., are, disappointed, The, and, ti...","{fishermen, ., are, disappointed, The, and, ti...","{fishermen, ., are, disappointed, The, and, ti...","{fishermen, ., are, disappointed, The, and, ti...",1,1,1


Conclusion: By looking at the above table we can conclude that compared to previous algorithms this has given a pearson of 0.33 as compared to senses which was 0.43

Prior to this a dictionary was created and replaced with the word, however, not all occurences of the word were replaced. This is why using RegEx, we were able to replace all instances of the corresponding NEC-Word pairs. 

Overall, NEC is an interesting method to compare sentences but it would highly depend on the type of context the corpus comes from. As seen in this laboratory, and compared to the previous ones it would not be the first choice as the jaccard distances fluctuate too much between original sentence and NEC replaced sentence.